<h1 style="text-align: center; font-weight: bold">Training - Pipeline</h1>

---

# 1 - Notebook Setup

## 1.1 - Imports

In [1]:
# system
from IPython.display import clear_output

import sys
sys.path[0] = sys.path[0].replace('notebooks', '')

# visualization
import matplotlib.pyplot as plt
import tqdm

# model
from model import ChatBot

import torch.optim as optim

# data manipulation
from torch.utils.data import random_split, Dataset, DataLoader
from pathlib import Path

import torch.nn as nn
import pandas as pd
import numpy as np
import torch
import json
import os

/home/mtxsouza/.cache/pypoetry/virtualenvs/chatbot-41bQ2a1z-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1.2 - Dataset path

In [2]:
try:
    from google.colab import drive 
    drive.mount('/content/gdrive')
    df_path = Path('/content/gdrive/MyDrive/')
    figure_path = Path('/content')
    weight_path = Path('/content/gdrive/MyDrive/')
    ENV = 'Google Colab'
except Exception as error:
    df_path = Path(os.getcwd().replace('notebooks', 'data/processed'))
    figure_path = Path(os.getcwd().replace('notebooks', 'figures'))
    weight_path = Path(os.getcwd().replace('notebooks', 'model')).joinpath('weights')
    weight_path.mkdir(exist_ok=True)
    ENV = 'Local Machine'

print(f'[\033[1;33mENVIRONMENT\033[0m]: {ENV}')
print(f'Dataset located in {df_path}')

[ENVIRONMENT]: Local Machine
Dataset located in /home/mtxsouza/projects/ChatBot/data/processed


## 1.3 - Device (GPU|CPU)

In [3]:
device = torch.device(device='cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    gpu_available = '\033[1;32mAvailable\033[0m'
else:
    gpu_available = '\033[1;31mNot Available\033[0m'
print(f'[\033[1;36mGPU\033[0m]: {gpu_available}')

[GPU]: Available


# 2 - Custom Dataset

## 2.1 - Loading dataset

In [4]:
data_frame = pd.read_csv(df_path.joinpath('cleaned_dataframe.csv'), encoding='utf-8', delimiter=",", on_bad_lines='skip')
with open(file=df_path.joinpath('data_info.json'), mode='r') as json_file:
    json_content = json.load(fp=json_file)
    word_dict = json_content['word_dict']
    question_len = json_content['question_length']
    answer_len = json_content['answer_length']
    vocabulary = json_content['num_vocabulary']
    word_dict_num = {v: k for k, v in word_dict.items()}

In [5]:
data_frame.head(5)

,question,answers
0,first work generally recognized artificial int...,warren mcculloch and walter pitts 1943
1,sources drawn formation first work generally r...,knowledge of the basic physiology and function...
2,created hebbian learning rule,donald hebb 1949
3,first neural network built,1950
4,first neural network called,the snarc


## 2.2 - Splitting dataset

In [6]:
cleaned_dataframe_np = np.asarray(a=data_frame)

split_size = [0.8, 0.1, 0.1]
train_split, test_split, valid_split = random_split(dataset=cleaned_dataframe_np, lengths=split_size)

In [7]:
train_df = cleaned_dataframe_np[train_split.indices]
test_df = cleaned_dataframe_np[test_split.indices]
valid_df = cleaned_dataframe_np[valid_split.indices]

In [8]:
for df_name, data in zip(['train', 'test', 'valid'], [train_df, test_df, valid_df]):
    print(f'{df_name:5}: {data.shape}')

train: (3377, 2)
test : (422, 2)
valid: (422, 2)


## 2.3 - Creating custom dataset class

In [9]:
class CustomDataset(Dataset):

    def __init__(self, dataset: np.ndarray, word_dict: dict, voc_size: int, max_len_in: int = 30, max_len_out: int = 500) -> None:
        
        self.x = dataset[:,0]
        self.y = dataset[:,1]

        self.wd = word_dict
        self.vs = voc_size
        self.mli = max_len_in
        self.mlo = max_len_out
    
    def __len__(self) -> int:
        return len(self.x)
    
    def __getitem__(self, index: int) -> torch.Tensor:
        
        x = self.x[index]
        y = self.y[index]

        x, y = self.gen_data(x=x, y=y)
        x = torch.from_numpy(x)
        y = torch.from_numpy(y)

        return x, y
    
    def gen_data(self, x: np.ndarray, y: np.ndarray) -> np.ndarray:

        x_data = []
        y_data = []

        x_data.append(self.wd['<start>'])
        for word in x.split(' '):
            x_data.append(self.wd[word])
        x_data.append(self.wd['<stop>'])
        while len(x_data) < self.mli:
            x_data.append(self.wd['<pad>'])
        x_data = np.asarray(a=x_data)

        y_data.append(self.wd['<start>'])
        for word in y.split(' '):
            y_data.append(self.wd[word])
        y_data.append(self.wd['<stop>'])
        while len(y_data) < self.mlo:
            y_data.append(self.wd['<pad>'])
        y_data = np.asarray(a=y_data)

        return x_data, y_data

In [10]:
train_custom_df = CustomDataset(dataset=train_df, word_dict=word_dict, voc_size=vocabulary, max_len_in=question_len, max_len_out=answer_len)
test_custom_df = CustomDataset(dataset=test_df, word_dict=word_dict, voc_size=vocabulary, max_len_in=question_len, max_len_out=answer_len)
valid_custom_df = CustomDataset(dataset=valid_df, word_dict=word_dict, voc_size=vocabulary, max_len_in=question_len, max_len_out=answer_len)

In [11]:
for x, y in test_custom_df:
    print(f'X: {x.size()}')
    print(f'Y: {y.size()}')
    break

X: torch.Size([30])
Y: torch.Size([400])


# 3 - Model

In [12]:
model = ChatBot(in_f=question_len, out_shape=(vocabulary, answer_len), word_dict=word_dict, device=device)

## 3.1 - Checking architecture

In [ ]:
build = model(
    torch.zeros(size=(2,question_len)).to(device=device),
    torch.zeros(size=(2,answer_len)).to(device=device)
)
build.size()

torch.Size([2, 5249, 399])

# 4 - Training

## 4.1 - Variables

### 4.1.1 - Training parameters

In [ ]:
epochs = 100
batch_size = 32
lr = 1e-3
early_stop = 5

n_plots = 10

### 4.1.2 - Initializing weights

In [ ]:
def _init_weights(module) -> None:
    if isinstance(module, (nn.Linear, nn.Embedding)):
        module.weight.data.normal_(mean=0, std=0.02)

In [ ]:
model.apply(_init_weights)

ChatBot(
  (l_h): Linear(in_features=30, out_features=512, bias=True)
  (l_c): Linear(in_features=30, out_features=512, bias=True)
  (emb): Embedding(5249, 399)
  (lstm): LSTM(429, 512, batch_first=True)
  (lin): Linear(in_features=512, out_features=5249, bias=True)
  (drop): Dropout(p=0.3, inplace=False)
)

### 4.1.3 - Loss function

In [ ]:
loss_fn = nn.CrossEntropyLoss(ignore_index=word_dict['<pad>'])

### 4.1.4 - Optimizer

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=lr)

## 4.2 - Training methods

In [ ]:
def plot_train_loss(loss: list, val_loss: list, epoch: int) -> None:
    
    fig, ax = plt.subplots(figsize=(10,4))

    for data, label in [(loss, 'train'), (val_loss, 'valid')]:
        try:
            ax.plot(np.arange(stop=epoch, step=epoch / len(data)), data, label=label)
        except:
            ax.plot(np.arange(stop=epoch, step=epoch / (len(data) - 0.5)), data, label=label)
    
    ax.set_xticks(np.arange(start=1, stop=epoch + 1))
    ax.set_title(label='Loss graph', fontdict={'size': 10})

    plt.legend(loc='upper right')
    plt.show()

## 4.3 - Creating dataloaders

In [ ]:
train_loader = DataLoader(dataset=train_custom_df, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_custom_df, batch_size=batch_size, shuffle=True)

## 4.4 - Training loop

In [ ]:
train_losses = []
valid_losses = []

best_loss = float('inf')
best_epoch = 'No saved yet'

previous_train = float('inf')
previous_valid = float('inf')

valid_x = []
valid_y = []
valid_yhat = []

overfitting = 0


for curr_epoch in range(1, epochs+1):

    print(f'Epoch: {curr_epoch}/{epochs} | Batch: {batch_size} | Learing Rate: {lr} | Loss Function: {loss_fn.__class__.__name__} | Optimizer: {optimizer.__class__.__name__}')
    print(f'Overtting: {overfitting}/{early_stop} | Best loss: {best_loss} - Last savement: {best_epoch}')

    if overfitting == early_stop:
        print('\033[1;31mOVERFITTING\033[0m')
        break

    # training
    train_iter = tqdm.tqdm(iterable=train_loader)
    train_loss = 0
    model.train()
    for x, y in train_iter:

        x = x.to(device=device)
        y = y.to(device=device)

        yhat = model(x, y.long())

        loss = loss_fn(yhat.view(-1, vocabulary), y[:,1:].reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # saving train results
        train_losses.append(loss.cpu().detach().numpy())
        train_iter.set_description(desc=f'Loss: {loss.item()}')
        train_loss += loss.item()

        del x, y, yhat, loss

    train_loss /= len(train_loader)

    # validing
    valid_x.clear()
    valid_y.clear()
    valid_yhat.clear()
    valid_iter = tqdm.tqdm(iterable=valid_loader)
    valid_loss = 0
    model.eval()
    with torch.no_grad():
        for x, y in valid_iter:

            x = x.to(device=device)
            y = y.to(device=device)

            yhat = model(x, y.long())

            loss = loss_fn(yhat.view(-1, vocabulary), y[:,1:].reshape(-1))

            # saving valid results
            valid_losses.append(loss.cpu().detach().numpy())
            valid_iter.set_description(desc=f'Valid loss: {loss.item()}')
            valid_loss += loss.item()

            # saving samples
            if len(valid_x) < n_plots:
                if np.random.random() > .7:

                    rand_idx = np.random.choice(a=x.size(0), size=1).item()

                    valid_x.append(x[rand_idx,:].unsqueeze(0).cpu())
                    valid_y.append(y[rand_idx,:].unsqueeze(0).cpu())
                    valid_yhat.append(yhat[rand_idx,:,:].unsqueeze(0).cpu().argmax(dim=1))
            
            del x, y, yhat, loss

    valid_loss /= len(valid_loader)
    
    # saving model
    if best_loss > valid_loss:
        best_loss = valid_loss
        best_epoch = curr_epoch
        torch.save(obj=model.state_dict(), f=weight_path.joinpath('chatbot_weights.pt'))
    
    # checking overfits
    if previous_train > train_loss and previous_valid < valid_loss:
        overfitting += 1
    else:
        if overfitting > 0:
            overfitting -= 1
    previous_train = train_loss
    previous_valid = valid_loss
    
    # plotting graphs
    clear_output()
    plot_train_loss(loss=train_losses, val_loss=valid_losses, epoch=curr_epoch)
    for x, y, yhat in zip(valid_x, valid_y, valid_yhat):
        for content, name in zip((x[0], y[0], yhat[0]), ('question', 'answer', 'predict')):
            text = []
            for values in content:
                word = word_dict_num[values.item()].replace('<start>', '').replace('<stop>', '').replace('<pad>', '')
                if len(word):
                    text.append(word)
            text = ' '.join(text).strip()
            print(f'{name:10}: {text}')
        print('-' * 50)

Epoch: 1/100 | Batch: 16 | Learing Rate: 0.001 | Loss Function: CrossEntropyLoss | Optimizer: Adam
Overtting: 0/5 | Best loss: inf - Last savement: No saved yet


Loss: 7.893790245056152:   1%|          | 2/212 [00:06<11:35,  3.31s/it]


KeyboardInterrupt: 